In [ ]:
# workflow for single book
from nlppln import WorkflowGenerator
#cwl_working_dir = '/home/dafne/cwl-working-dir/'
cwl_working_dir = '/home/jvdzwaan/cwl-working-dir/'

In [ ]:
with WorkflowGenerator(working_dir=cwl_working_dir) as wf_sub:
    wf_sub.load(steps_dir='../adhtools/cwl/')
    wf_sub.load(steps_dir='../java/cwl/')
    print(wf_sub.list_steps())
    
    analyzer = wf_sub.add_input(analyzer='string', default='Alkhalil')
    book = wf_sub.add_input(book='File')
    cp = wf_sub.add_input(cp='string')
    
    metadata, txt_dir = wf_sub.txt2safar_input(in_file=book)
    analyzed_files = wf_sub.SafarAnalyze(in_dir=txt_dir, analyzer=analyzer, cp=cp)
    safar_output_dir = wf_sub.safar_add_metadata(in_files=analyzed_files, meta_in=metadata)
    
    wf_sub.add_outputs(safar_output_dir=safar_output_dir)
    
    wf_sub.save('../adhtools/cwl/safar-workflow.cwl', wd=True, relative=False)

In [ ]:
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    wf.load(step_file='https://raw.githubusercontent.com/arabic-digital-humanities/BlackLabIndexer-docker/master/blacklabindexer.cwl')
    
    print(wf.list_steps())
    
    analyzer = wf.add_input(analyzer='string', default='Alkhalil')
    book = wf.add_input(book='File')
    cp = wf.add_input(cp='string')
    index_name = wf.add_input(index_name='string', default='corpus')
    action = wf.add_input(action='string', default='create')
    index_format = wf.add_input(index_format='string', default='safar-analyzer')
    text_direction = wf.add_input(text_direction='string', default='rtl')
    content_viewable = wf.add_input(content_viewable='boolean', default=True)
    
    safar_output_dir = wf.safar_workflow(analyzer=analyzer, book=book, cp=cp) #, scatter='book'
    indexed = wf.blacklabindexer(action=action, 
                                 index_format=index_format, 
                                 index_name=index_name, 
                                 in_dir=safar_output_dir, 
                                 text_direction=text_direction, 
                                 content_viewable=content_viewable)
    
    wf.add_outputs(indexed=indexed)
    
    wf.add_outputs(safar_output_dir=safar_output_dir)
    
    wf.save('../adhtools/cwl/process.cwl', wd=True, relative=False)
    
    # select txt files -> ls?
    # divide and extract metadata -> safar2text_and_metadata
    # save to dir -> save_to_dir
    # safar analyze -> SafarAnalyze.java -> maak dat ie uit dir leest
    # add metadata -> safar_add_metadata
    # index with blacklab

In [ ]:
# Now the scattered version
with WorkflowGenerator(working_dir=cwl_working_dir) as wf:
    wf.load(steps_dir='../adhtools/cwl/')
    wf.load(steps_dir='../java/cwl/')
    wf.load(step_file='https://raw.githubusercontent.com/arabic-digital-humanities/BlackLabIndexer-docker/master/blacklabindexer.cwl')
    
    print(wf.list_steps())
    
    analyzer = wf.add_input(analyzer='string', default='Alkhalil')
    in_dir = wf.add_input(in_dir='Directory')
    cp = wf.add_input(cp='string')
    index_name = wf.add_input(index_name='string', default='corpus')
    action = wf.add_input(action='string', default='create')
    index_format = wf.add_input(index_format='string', default='safar-analyzer')
    text_direction = wf.add_input(text_direction='string', default='rtl')
    content_viewable = wf.add_input(content_viewable='boolean', default=True)
    
    books = wf.ls(in_dir=in_dir)
    safar_output_dirs = wf.safar_workflow(analyzer=analyzer, book=books, cp=cp, scatter='book', scatter_method='dotproduct')
    merged_dir = wf.gather_dirs(in_dirs=safar_output_dirs)
    indexed = wf.blacklabindexer(action=action, 
                                 index_format=index_format, 
                                 index_name=index_name, 
                                 in_dir=merged_dir, 
                                 text_direction=text_direction, 
                                 content_viewable=content_viewable)
    wf.add_outputs(indexed=indexed)
    
    wf.add_outputs(safar_output_dirs=safar_output_dirs)
    wf.add_outputs(merged_dir=merged_dir)
    
    wf.save('../adhtools/cwl/process_dir.cwl', wd=True, relative=False)